In [1]:
!nvidia-smi

Tue May 28 01:53:32 2024       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.147.05   Driver Version: 525.147.05   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A100-SXM...  Off  | 00000000:07:00.0 Off |                    0 |
| N/A   30C    P0    57W / 400W |   5640MiB / 40960MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
|   1  NVIDIA A100-SXM...  On   | 00000000:0A:00.0 Off |                    0 |
| N/A   

In [2]:
import torch


# Function to list available GPUs and select one
def select_device():
    if torch.cuda.is_available():
        print("Available GPUs:")
        for i in range(torch.cuda.device_count()):
            print(f"{i}: {torch.cuda.get_device_name(i)}")
        device_id = int(input("Select GPU by entering the device ID (default 0): ") or 0)
        if device_id < torch.cuda.device_count():
            print(f"Using GPU: {torch.cuda.get_device_name(device_id)}")
            return torch.device(f"cuda:{device_id}")
        else:
            print(f"Invalid device ID. Using GPU: {torch.cuda.get_device_name(0)}")
            return torch.device("cuda:0")
    else:
        print("No GPU available. Using CPU.")
        return torch.device("cpu")

# Select the device
device = select_device()

Available GPUs:
0: NVIDIA A100-SXM4-40GB
1: NVIDIA A100-SXM4-40GB
2: NVIDIA A100-SXM4-40GB
3: NVIDIA A100-SXM4-40GB
4: NVIDIA A100-SXM4-40GB
5: NVIDIA A100-SXM4-40GB
6: NVIDIA A100-SXM4-40GB
7: NVIDIA A100-SXM4-40GB
Select GPU by entering the device ID (default 0): 0
Using GPU: NVIDIA A100-SXM4-40GB


In [3]:
from transformers import RobertaForSequenceClassification, RobertaTokenizer
from torch.utils.data import DataLoader, Dataset
import torch

# Define the model and tokenizer for RoBERTa large
model_name = "FacebookAI/roberta-large"
tokenizer = RobertaTokenizer.from_pretrained(model_name)
model = RobertaForSequenceClassification.from_pretrained(model_name, num_labels=3)  # Assuming 3 labels for the task



/opt/anaconda3/lib/python3.11/site-packages/transformers/utils/generic.py:260: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at FacebookAI/roberta-large and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.weight', 'classifier.out_proj.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [4]:
import pandas as pd

# MNLI train dataset

df_mnli_train = pd.read_csv('MNLI/mnli_train.csv')

In [5]:
df_mnli_train 

,premise,hypothesis,label,idx
0,Conceptually cream skimming has two basic dime...,Product and geography are what make cream skim...,1,0
1,you know during the season and i guess at at y...,You lose the things to the following level if ...,0,1
2,One of our number will carry out your instruct...,A member of my team will execute your orders w...,0,2
3,How do you know? All this is their information...,This information belongs to them.,0,3
4,yeah i tell you what though if you go price so...,The tennis shoes have a range of prices.,1,4
...,...,...,...,...
392697,"Clearly, California can - and must - do better.",California cannot do any better.,2,392697
392698,It was once regarded as the most beautiful str...,So many of the original buildings had been rep...,1,392698
392699,Houseboats are a beautifully preserved traditi...,The tradition of houseboats originated while t...,0,392699
392700,Obituaries fondly recalled his on-air debates ...,The obituaries were beautiful and written in k...,1,392700


In [6]:
df_mnli_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 392702 entries, 0 to 392701
Data columns (total 4 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   premise     392702 non-null  object
 1   hypothesis  392662 non-null  object
 2   label       392702 non-null  int64 
 3   idx         392702 non-null  int64 
dtypes: int64(2), object(2)
memory usage: 12.0+ MB


In [7]:
def clean_text_and_label_data(df):
    # Drop any rows with NaN values
    df = df.dropna()
    
    # Ensure text columns are of string type and labels are of integer type using .loc
    df.loc[:, 'premise'] = df['premise'].astype(str)
    df.loc[:, 'hypothesis'] = df['hypothesis'].astype(str)
    df.loc[:, 'label'] = df['label'].astype(int)
    
    return df

# Clean the entire dataset first
df_mnli_train = clean_text_and_label_data(df_mnli_train)


# Check for any NaN values across the entire DataFrame
nan_check = df_mnli_train.isna().any().any()

# Print the result to see if there are any NaN values left
if nan_check:
    print("There are still NaN values in the DataFrame.")
else:
    print("No NaN values found in the DataFrame.")


No NaN values found in the DataFrame.


In [8]:
from sklearn.model_selection import train_test_split

# Now perform the train-test split
train_df, test_df = train_test_split(df_mnli_train, test_size=0.1, random_state=42)

# Verifying the size of the datasets
print(f"Training set size: {train_df.shape}")
print(f"Test set size: {test_df.shape}")

Training set size: (353395, 4)
Test set size: (39267, 4)


In [9]:
import numpy as np

# Function to tokenize and calculate the length of tokenized sentences
def calculate_token_lengths(texts, tokenizer):
    token_lengths = []
    for text in texts:
        tokens = tokenizer.encode(text, add_special_tokens=True)  # add_special_tokens=True for [CLS] and [SEP]
        token_lengths.append(len(tokens))
    return token_lengths

# Concatenate premises and hypotheses for length calculation
all_texts = pd.concat([df_mnli_train['premise'], df_mnli_train['hypothesis']]).dropna()

# Calculate token lengths
token_lengths = calculate_token_lengths(all_texts, tokenizer)

# Basic statistics about token lengths
max_length = np.max(token_lengths)
avg_length = np.mean(token_lengths)
percentile_95 = np.percentile(token_lengths, 95)  # 95th percentile

print(f"Maximum token length: {max_length}")
print(f"Average token length: {avg_length}")
print(f"95th percentile of token lengths: {percentile_95}")


Maximum token length: 408
Average token length: 20.335832089685276
95th percentile of token lengths: 46.0


In [10]:
import torch
import logging

logging.disable(logging.WARNING)

def tokenize_and_prepare_inputs(df, tokenizer, max_length=50):
    # Tokenize, pad, and create attention masks
    # 'max_length' is a parameter you can tune depending on the maximum length you observed in your dataset analysis
    tokenized_inputs = tokenizer(
        df['premise'].tolist(),
        df['hypothesis'].tolist(),
        max_length=max_length,
        padding='max_length',
        truncation=True,
        return_tensors="pt"
    )
    
    # Extracting input_ids and attention masks from the tokenized inputs
    input_ids = tokenized_inputs['input_ids']
    attention_masks = tokenized_inputs['attention_mask']
    
    return input_ids, attention_masks

# Tokenize training data
train_input_ids, train_attention_masks = tokenize_and_prepare_inputs(train_df, tokenizer)

# Tokenize validation (test) data
validation_input_ids, validation_attention_masks = tokenize_and_prepare_inputs(test_df, tokenizer)

# Convert labels to tensors
train_labels = torch.tensor(train_df['label'].values)
validation_labels = torch.tensor(test_df['label'].values)

# Print some details to verify everything is as expected
print("Training input IDs shape:", train_input_ids.shape)
print("Validation input IDs shape:", validation_input_ids.shape)
print("Training labels shape:", train_labels.shape)
print("Validation labels shape:", validation_labels.shape)


Training input IDs shape: torch.Size([353395, 50])
Validation input IDs shape: torch.Size([39267, 50])
Training labels shape: torch.Size([353395])
Validation labels shape: torch.Size([39267])


In [11]:
from torch.utils.data import DataLoader, TensorDataset

# Assuming you have train_input_ids, train_attention_masks, train_labels,
# validation_input_ids, validation_attention_masks, validation_labels already defined

# Create TensorDataset instances
train_dataset = TensorDataset(train_input_ids, train_attention_masks, train_labels)
validation_dataset = TensorDataset(validation_input_ids, validation_attention_masks, validation_labels)

# Define batch size
batch_size = 32  # You can adjust this based on your GPU memory availability and model complexity

# Create DataLoader instances
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
validation_loader = DataLoader(validation_dataset, batch_size=batch_size, shuffle=False)

# Print to confirm setup
print(f"Train DataLoader setup with {len(train_loader)} batches of size {batch_size}.")
print(f"Validation DataLoader setup with {len(validation_loader)} batches of size {batch_size}.")


Train DataLoader setup with 11044 batches of size 32.
Validation DataLoader setup with 1228 batches of size 32.


In [12]:
import torch
from torch.optim import AdamW 
from torch.optim.lr_scheduler import CyclicLR
from tqdm.auto import tqdm
import os

model.to(device)

# Optimizer setup
optimizer = AdamW(model.parameters(), lr=1e-5, weight_decay=1e-6)

# CLR Scheduler setup
scheduler = CyclicLR(optimizer, base_lr=5e-6, max_lr=1e-5, 
                     step_size_up=len(train_loader)//2, step_size_down=len(train_loader)//2,
                     mode='triangular', cycle_momentum=False)

# Ensure the save directory exists
save_path = '/storage/data/st1070263'


# Training and validation functions
def train_model(model, train_loader, validation_loader, optimizer, scheduler, epochs=3):
    steps_per_epoch = len(train_loader)
    for epoch in range(epochs):
        model.train()
        train_loss = 0
        correct = 0
        total = 0
        progress_bar = tqdm(train_loader, desc=f'Epoch {epoch + 1}/{epochs}', unit='batch')

        for batch_index, (input_ids, attention_masks, labels) in enumerate(progress_bar):
            input_ids, attention_masks, labels = input_ids.to(device), attention_masks.to(device), labels.to(device)
            model.zero_grad()
            outputs = model(input_ids=input_ids, attention_mask=attention_masks, labels=labels)
            loss = outputs.loss
            logits = outputs.logits

            loss.backward()
            optimizer.step()
            scheduler.step()  # Update the learning rate after each batch

            train_loss += loss.item()
            _, predicted = torch.max(logits, dim=1)
            correct += (predicted == labels).sum().item()
            total += labels.size(0)
            accuracy = 100. * correct / total
            lr = scheduler.get_last_lr()[0]

            progress_bar.set_postfix(loss=train_loss/(batch_index + 1), accuracy=f'{accuracy:.2f}%', lr=f'{lr:.6f}')

            # Save snapshot at the end of each cycle
            current_step = epoch * steps_per_epoch + batch_index
            if (current_step + 1) % scheduler.total_size == 0:
                torch.save(model.state_dict(), f'{save_path}/roberta_large_mnli_cycle_{current_step + 1}.pth')
                print(f"Snapshot saved at step {current_step + 1}")

        validate_model(model, validation_loader, device)

def validate_model(model, validation_loader, device):
    model.eval()
    val_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for input_ids, attention_masks, labels in tqdm(validation_loader, desc='Validating', leave=False, unit='batch'):
            input_ids, attention_masks, labels = input_ids.to(device), attention_masks.to(device), labels.to(device)
            outputs = model(input_ids=input_ids, attention_mask=attention_masks, labels=labels)
            loss = outputs.loss
            logits = outputs.logits
            val_loss += loss.item()
            _, predicted = torch.max(logits, dim=1)
            correct += (predicted == labels).sum().item()
            total += labels.size(0)

    val_accuracy = 100. * correct / total
    print(f'\nValidation Loss: {val_loss / len(validation_loader):.4f} | Validation Accuracy: {val_accuracy:.2f}%')

# Start training
train_model(model, train_loader, validation_loader, optimizer, scheduler, epochs=5)


Epoch 1/5:   0%|          | 0/11044 [00:00<?, ?batch/s]

Snapshot saved at step 11044


Validating:   0%|          | 0/1228 [00:00<?, ?batch/s]


Validation Loss: 0.3235 | Validation Accuracy: 87.48%


Epoch 2/5:   0%|          | 0/11044 [00:00<?, ?batch/s]

...


Validation Loss: 0.4127 | Validation Accuracy: 87.56%


Epoch 5/5:   0%|          | 0/11044 [00:00<?, ?batch/s]

Snapshot saved at step 55220


Validating:   0%|          | 0/1228 [00:00<?, ?batch/s]


Validation Loss: 0.4747 | Validation Accuracy: 87.35%
